# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from JamesWilliams_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
clean_cities_data = pd.read_csv("cities.csv", index_col=0)
clean_cities_data

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,abha,18.22,42.51,59.00,35,0,2.39,SA,1604009482
1,lac du bonnet,50.25,-96.06,28.00,98,75,5.99,CA,1604009483
2,san rafael,-34.62,-68.33,66.97,25,1,13.96,AR,1604009484
3,fairbanks,64.84,-147.72,19.00,78,90,0.36,US,1604009373
4,nikolskoye,59.70,30.79,50.00,93,90,4.47,RU,1604009487
...,...,...,...,...,...,...,...,...,...
543,itarema,-2.92,-39.92,79.02,77,2,16.46,BR,1604010163
544,hofn,64.25,-15.21,46.00,80,100,12.64,IS,1604010165
545,havre-saint-pierre,50.23,-63.60,30.54,87,62,17.67,CA,1604010166
546,srednekolymsk,67.45,153.68,3.15,94,100,0.98,RU,1604010167


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

locations = clean_cities_data[['Lat', 'Lon']].astype(float)
humidity = clean_cities_data["Humidity"].astype(float)

In [5]:
fig = gmaps.figure(center=(0,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
temp70_weather_df = clean_cities_data.loc[clean_cities_data['Max Temp'] > 70]
temp80_weather_df = temp70_weather_df.loc[temp70_weather_df['Max Temp'] < 80]
windsp_weather_df = temp80_weather_df.loc[temp80_weather_df['Wind Speed'] > 10]
best_weather_df = windsp_weather_df.loc[windsp_weather_df['Cloudiness'] == 0]
best_weather_df = best_weather_df.dropna()
best_weather_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
200,buin,-33.73,-70.75,78.8,19,0,10.29,CL,1604009734
432,tabora,-5.02,32.80,78.8,47,0,11.41,TZ,1604010020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = best_weather_df
hotel_df['Hotel Name'] = ""

hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
200,buin,-33.73,-70.75,78.8,19,0,10.29,CL,1604009734,
432,tabora,-5.02,32.80,78.8,47,0,11.41,TZ,1604010020,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lon']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
    

hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
200,buin,-33.73,-70.75,78.8,19,0,10.29,CL,1604009734,Linderos B&B
432,tabora,-5.02,32.80,78.8,47,0,11.41,TZ,1604010020,Tabora Belmonte Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
                             
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))